In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# from permutations import kendalltau_dist
# from permutations import aggregate_parity_new, aggregate_kemeny_new
# from permutations import gen_groups

In [3]:
animelist = pd.read_csv("anime_data/animelist.csv")

In [4]:
## watching status 2 == completed

In [5]:
completed = animelist[animelist.watching_status == 2].reset_index(drop=True)

In [6]:
completed.user_id.nunique()

320781

In [7]:
completed.anime_id.nunique()

16905

In [8]:
animes = pd.read_csv("anime_data/anime.csv")

In [9]:
top_anime_ids = completed.groupby("anime_id").size().sort_values(ascending=False).head(15)
top_animes = animes[animes['MAL_ID'].isin(top_anime_ids.index)]
top_animes = top_animes.set_index('MAL_ID')
top_animes = top_animes.loc[top_anime_ids.index].reset_index()

In [10]:
def get_users_rating_all_top_anime(completed, n_top_anime=20):
    """
    Find users who have rated all of the top N anime and return their ratings.
    
    Parameters:
    -----------
    completed : DataFrame
        DataFrame containing anime ratings with user_id and anime_id columns
    n_top_anime : int, default=20
        Number of top anime to consider
        
    Returns:
    --------
    final_subset : DataFrame
        Subset of ratings from users who rated all top N anime
    top_anime_info : DataFrame
        Information about the top N anime
    """
    # Get the top N anime_ids
    top_anime_ids = completed.groupby("anime_id").size().sort_values(ascending=False).head(n_top_anime).index.tolist()
    
    # Filter to only include ratings for these top anime
    top_anime_ratings = completed[completed['anime_id'].isin(top_anime_ids)]
    
    # Count how many of the top anime each user has rated
    user_rating_counts = top_anime_ratings.groupby('user_id')['anime_id'].nunique()
    
    # Find users who have rated all top N anime
    complete_users = user_rating_counts[user_rating_counts == n_top_anime].index.tolist()
    
    # Get the final subset of ratings from users who rated all top N anime
    final_subset = top_anime_ratings[top_anime_ratings['user_id'].isin(complete_users)]
    
    # Print some statistics
    print(f"Number of users who rated all top {n_top_anime} anime: {len(complete_users)}")
    print(f"Total number of ratings in the final subset: {len(final_subset)}")
    return final_subset


In [11]:
# d=16
# top100 = completed.groupby("anime_id").size().sort_values(ascending=False).head(100).index.tolist()
# random_subset = random.sample(top100, d)
# random_subset

In [12]:
# top100 = completed.groupby("anime_id").size().sort_values(ascending=False).head(25).index.tolist()
# random_subset = random.sample(top100, d)
# top_anime_ratings = completed[completed['anime_id'].isin(random_subset)]

# # Count how many of the top anime each user has rated
# user_rating_counts = top_anime_ratings.groupby('user_id')['anime_id'].nunique()

# # Find users who have rated all top N anime
# complete_users = user_rating_counts[user_rating_counts == d].index.tolist()

# # Get the final subset of ratings from users who rated all top N anime
# final_subset = top_anime_ratings[top_anime_ratings['user_id'].isin(complete_users)]

# # Print some statistics
# print(f"Number of users who rated all top {d} anime: {len(complete_users)}")
# print(f"Total number of ratings in the final subset: {len(final_subset)}")

In [13]:

final_subset = get_users_rating_all_top_anime(completed, n_top_anime=16)


Number of users who rated all top 16 anime: 8703
Total number of ratings in the final subset: 139248


In [14]:
final_subset.rating.value_counts()

rating
9     33638
8     31644
10    29304
7     17565
0     12475
6      7362
5      3484
4      1908
3      1016
2       466
1       386
Name: count, dtype: int64

In [15]:
final_subset.drop(columns=['watching_status', 'watched_episodes']).to_csv('anime_data/fairness_experiment.csv')

### Experiment

In [16]:
rating_df = pd.read_csv('anime_data/fairness_experiment.csv')
rating_df.shape

(139248, 4)

In [17]:
rating_df.user_id.nunique() 

8703

In [18]:
def transform_df_to_rankings(df):
    """
    Transform a DataFrame with columns ['user_id', 'anime_id', 'rating']
    to a list of lists, where each inner list contains anime_ids ranked by rating for a user (is a partial list).
    Breaks ties arbitrarily by adding small random noise to ratings.
    
    Args:
        df: DataFrame with columns ['user_id', 'anime_id', 'rating']
        
    Returns:
        List of lists, where each inner list is a ranking of anime_ids for a user
    """
    df_copy = df.copy()
    df_copy['rating_with_noise'] = df_copy['rating'] + np.random.uniform(-0.5, 0.5, size=len(df_copy))
    user_rankings = []
    
    for user_id, group in df_copy.groupby('user_id'):
        # Sort by rating_with_noise in descending order (highest rating first)
        sorted_group = group.sort_values('rating_with_noise', ascending=False)
        anime_ranking = sorted_group['anime_id'].tolist()
        user_rankings.append(anime_ranking)
    
    return user_rankings

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time

def create_uniform_coreset(all_rankings, coreset_size):
    """
    Create a uniform random coreset of rankings by sampling with replacement.
    """
    indices = np.random.choice(len(all_rankings), size=coreset_size, replace=True)
    uniform_coreset = [all_rankings[i] for i in indices]
    return uniform_coreset

In [20]:
# import numpy as np

# def convert_rankings_to_indices(rankings, ground_set):
#     element_to_index = {element: idx for idx, element in enumerate(ground_set)}
#     index_to_element = {idx: element for idx, element in enumerate(ground_set)}
#     indexed_rankings = []
#     for ranking in rankings:
#         indexed_ranking = [element_to_index[element] for element in ranking]
#         indexed_rankings.append(indexed_ranking)
#     return np.array(indexed_rankings), element_to_index, index_to_element

# def convert_indices_to_rankings(indexed_rankings, index_to_element):
#     """
#     Convert numpy array of indices back to original rankings.
    
#     Args:
#         indexed_rankings: numpy.ndarray of indices
#         index_to_element: dict mapping indices to original elements
    
#     Returns:
#         list: List of rankings with indices replaced by original elements
#     """
#     original_rankings = []
#     for ranking in indexed_rankings:
#         original_ranking = [index_to_element[idx] for idx in ranking]
#         original_rankings.append(original_ranking)
    
#     return original_rankings

## Experiment 2

In [22]:
from fair_rank_aggregation import kemeny_rank_aggregation, kemeny_rank_aggregation_with_parity, compute_kendall_tau_cost, create_group_assignment

In [23]:
rankings = transform_df_to_rankings(rating_df)

In [24]:
ground_set = set(rankings[0])

In [29]:
def run_fairness_experiment(ground_set, all_rankings, prob_values, delta=0.1, coreset_size=None, repetitions=20):
    aggregated = kemeny_rank_aggregation(rankings, ground_set)
    found = False
    while not found:
        groups_list = []
        for p in prob_values:
            group = create_group_assignment(aggregated, p)
            if any(existing_group == group for existing_group in groups_list):
                break
            groups_list.append(group)
        found = True
    print(groups_list)
    return _run_fairness_experiment(ground_set, all_rankings, groups_list, prob_values, delta, coreset_size, repetitions)
        


In [40]:
def _run_fairness_experiment(ground_set, all_rankings, groups_list, prob_values, delta=0.1, coreset_sizes=None, repetitions=20):
    """
    Run experiment to evaluate costs with fairness constraints across different coreset sizes.

    Args:
        n_candidates: Number of candidates in each ranking
        all_rankings: The full dataset of rankings
        groups_values: Group assignments for candidates corresponding to each p value
        p_values: List of p values (constraint parameters) from 0.5 to 1
        delta: Delta parameter for aggregate_parity_new
        coreset_sizes: List of coreset sizes to evaluate

    Returns:
        results_dict: Dictionary containing experiment results
    """
    if coreset_sizes is None:
        coreset_sizes = np.arange(100, 1050, 50)

    results = {
        'coreset_sizes': coreset_sizes,
        'p_values': prob_values,
        'full_costs': {p: None for p in prob_values},
        'optimal_rankings': {p: None for p in prob_values},
        'costs': {p: [] for p in prob_values},
        'stds': {p: [] for p in prob_values}
    }

    # Calculate optimal solutions and costs for full dataset
    print("Computing baseline solutions using full dataset...")
    for i in tqdm(range(len(groups_list)), desc="Processing p values"):
        group = groups_list[i]
        p = prob_values[i]
        try:
            optimal_ranking = kemeny_rank_aggregation_with_parity(all_rankings, ground_set, group, delta)
            
            if optimal_ranking is not None:
                optimal_cost = compute_kendall_tau_cost(optimal_ranking, all_rankings)
                results['full_costs'][p] = optimal_cost
                results['optimal_rankings'][p] = optimal_ranking
            else:
                print(f"Failed to find optimal solution for p={p:.2f}")
                results['full_costs'][p] = np.nan
        except Exception as e:
            print(f"Error with p={p}: {e}")
            results['full_costs'][p] = np.nan

    # Calculate costs for coresets
    for size in tqdm(coreset_sizes, desc="Testing coreset sizes"):
        costs = {p: [] for p in prob_values}
        for _ in range(repetitions):
        # Create coreset
            coreset = create_uniform_coreset(all_rankings, size)
            for i in range(len(prob_values)):
                group = groups_list[i]
                p = prob_values[i]
                coreset_ranking= kemeny_rank_aggregation_with_parity(coreset, ground_set, group, delta)
                coreset_cost = compute_kendall_tau_cost(coreset_ranking, all_rankings)
                costs[p].append(coreset_cost)
        for p in prob_values:
            results['costs'][p].append(np.mean(costs[p]))
            results['stds'][p].append(np.std(costs[p]))
    return results

def plot_fairness_results_old(results):
    """
    Create a professional paper-quality plot with smooth lines showing costs for
    different fairness constraints across coreset sizes.
    """
    plt.figure(figsize=(10, 6))
    sns.set_theme(style="whitegrid")
    colors = ['#0173B2', '#DE8F05', '#029E73', '#D55E00', '#CC78BC', '#CA9161']

    
    # Plot coreset results with smooth lines
    for i, p in enumerate(results['p_values']):
        # Convert to numpy array for easier handling
        costs_array = np.array(results['costs'][p])
        print("array", costs_array)
        
        # Plot with smooth lines
        plt.plot(results['coreset_sizes'], costs_array, '-', 
                color=colors[i % len(colors)], 
                label=f"p={p:.2f} (Coreset)", 
                linewidth=2.5)
    
    # Add horizontal lines for full dataset results
    for i, p in enumerate(results['p_values']):
        if not np.isnan(results['full_costs'][p]):
            plt.axhline(y=results['full_costs'][p], linestyle='--', 
                      color=colors[i % len(colors)], 
                      label=f"p={p:.2f} (Full Dataset)", 
                      linewidth=2)
    
    # Improved styling
    plt.xlabel('Coreset Size', fontsize=14)
    plt.ylabel('Kendall Tau Cost', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.3)
    
    # Use log scale for y-axis to better visualize differences
    plt.yscale('log')
    
    # Add nice legend
    plt.legend(fontsize=12, frameon=True, framealpha=0.95, edgecolor='gray', fancybox=False)
    
    # Set x-axis ticks
    plt.xticks(results['coreset_sizes'][::4], fontsize=12)
    plt.yticks(fontsize=12)
    
    # Make the spines visible but subtle
    for spine in plt.gca().spines.values():
        spine.set_visible(True)
        spine.set_linewidth(0.5)
        spine.set_color('gray')
    
    plt.tight_layout()
    plt.savefig('fairness_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()


In [34]:
len(rankings)

8703

In [ ]:
final_subset = final_subset[['user_id', 'anime_id', 'rating']]
final_subset

In [ ]:
final_subset.columns
final_subset.watched_episodes.describe()

In [ ]:
final_subset[final_subset.rating==0]

In [ ]:
final_subset

In [38]:
def plot_fairness_results(results, num_repeats=20, std_devs=True):
    """
    Create a professional paper-quality plot showing relative error (%) for
    different fairness constraints across coreset sizes with regression lines and data points.
    
    Parameters:
    -----------
    results : dict
        Dictionary containing 'coreset_sizes', 'costs', 'full_costs', 'p_values' and 'stds'
    num_repeats : int
        Number of repetitions used to compute standard error
    std_devs : bool, default=True
        Whether to plot standard deviation bands
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    from scipy import stats
    
    plt.figure(figsize=(10, 6))
    sns.set_theme(style="whitegrid")
    colors = ['#0173B2', '#DE8F05', '#029E73', '#D55E00', '#CC78BC', '#CA9161']
    
    # Plot relative error with regression lines and data points
    for i, p in enumerate(results['p_values']):
        # Skip if no full cost or if it's zero (to avoid division by zero)
        if p not in results['full_costs'] or results['full_costs'][p] == 0:
            continue
            
        # Get the full dataset cost for this p value
        full_cost = results['full_costs'][p]
        
        # Convert costs to relative error percentage: ((coreset_cost - full_cost) / full_cost) * 100
        costs_array = np.array(results['costs'][p])
        rel_errors = ((costs_array - full_cost) / full_cost) * 100
        
        # Get standard deviation array and convert to standard error if needed
        if std_devs and 'stds' in results and p in results['stds']:
            stds_array = np.array(results['stds'][p])
            std_errors = (stds_array / np.sqrt(num_repeats)) / full_cost * 100
        
        # Calculate linear regression line if we have at least 2 points
        if len(results['coreset_sizes']) >= 2:
            # Create X range for regression line (extend slightly beyond the data)
            x_range = np.linspace(min(results['coreset_sizes']) * 0.95, 
                                 max(results['coreset_sizes']) * 1.05, 100)
            
            # Linear regression (1st degree polynomial)
            slope, intercept, r_value, p_value, std_err = stats.linregress(
                results['coreset_sizes'], rel_errors)
            line = slope * x_range + intercept
            
            # Plot the regression line
            plt.plot(x_range, line, '-', 
                    color=colors[i % len(colors)], 
                    label=f"p={p}", 
                    linewidth=2.5,
                    zorder=10)
                
            # Add confidence interval as a shaded area if requested
            if std_devs and 'stds' in results and p in results['stds']:
                plt.fill_between(results['coreset_sizes'], 
                                rel_errors - std_errors,
                                rel_errors + std_errors,
                                color=colors[i % len(colors)], 
                                alpha=0.15,
                                zorder=5)
        else:
            # Just plot a single point if only one data point
            plt.plot(results['coreset_sizes'], rel_errors, '-', 
                    color=colors[i % len(colors)], 
                    label=f"p={p:.2f}", 
                    linewidth=2.5,
                    zorder=10)
            
            # Add confidence interval if requested
            if std_devs and 'stds' in results and p in results['stds']:
                plt.fill_between(results['coreset_sizes'], 
                                rel_errors - std_errors,
                                rel_errors + std_errors,
                                color=colors[i % len(colors)], 
                                alpha=0.15,
                                zorder=5)
        
        # Add markers for the actual data points
        plt.scatter(results['coreset_sizes'], rel_errors, 
                   color=colors[i % len(colors)], 
                   s=50,  # Size of dots
                   zorder=15,  # Make sure dots are on top
                   edgecolor='white',  # White edge makes dots stand out
                   linewidth=1)
    
    # Add a horizontal line at y=0 (representing no difference from full dataset)
    plt.axhline(y=0, linestyle='-', color='black', linewidth=1, alpha=0.5, zorder=5)
    
    # Improved styling
    plt.xlabel('Coreset Size', fontsize=14)
    plt.ylabel('Relative Error (%)', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.3)
    
    # Determine number of columns for legend based on number of p_values
    ncol = 2
    if len(results['p_values']) > 6:
        ncol = 3
    
    # Add a multi-column legend in the upper left corner
    plt.legend(fontsize=11, frameon=True, framealpha=0.95,
              edgecolor='gray', fancybox=False,
              loc='lower right',  # Position in upper left
              bbox_to_anchor=(1, 0),  # Fine-tune position
              ncol=ncol)  # Multiple columns for more compact width
    
    # Set x-axis ticks
    if len(results['coreset_sizes']) > 4:
        plt.xticks(results['coreset_sizes'][::2], fontsize=12)
    else:
        plt.xticks(results['coreset_sizes'], fontsize=12)
    plt.yticks(fontsize=12)
    
    # Make the spines visible but subtle
    for spine in plt.gca().spines.values():
        spine.set_visible(True)
        spine.set_linewidth(0.5)
        spine.set_color('gray')
    
    plt.tight_layout()
    plt.savefig('figures/fairness_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return plt.gcf()  # Return the figure for further customization if needed

In [ ]:
repetitions = 20
start_time = time.time()
results = run_fairness_experiment(ground_set, rankings, [0.5,0.25,0.125], delta=0.1, coreset_size=None, repetitions=repetitions)
print(f"took {time.time() - start_time} seconds")
plot_fairness_results(results, repetitions)

[{5114: 0, 2904: 0, 32281: 0, 1575: 1, 1535: 1, 30276: 1, 199: 1, 16498: 1, 4224: 1, 19815: 0, 6547: 1, 31964: 0, 20: 1, 22319: 0, 10620: 0, 11757: 0}, {5114: 0, 2904: 1, 32281: 1, 1575: 0, 1535: 0, 30276: 0, 199: 0, 16498: 1, 4224: 0, 19815: 1, 6547: 0, 31964: 0, 20: 1, 22319: 1, 10620: 1, 11757: 1}, {5114: 0, 2904: 0, 32281: 0, 1575: 0, 1535: 0, 30276: 0, 199: 0, 16498: 0, 4224: 1, 19815: 1, 6547: 1, 31964: 1, 20: 1, 22319: 1, 10620: 1, 11757: 1}]
Computing baseline solutions using full dataset...


Testing coreset sizes:  42%|████████▊            | 8/19 [08:42<12:04, 65.89s/it]

In [ ]:
plot_fairness_results(results, repetitions)